Importando bibliotecas

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA

Importando Base de Dados

In [ ]:
df = pd.read_csv("../databases/ENEM_2023_FINAL_cat.csv")
display(df.sample(5))

Definindo funções que realizam testes de anova e manova

In [ ]:
def anova_test(df:pd.DataFrame, y_col:str, x_cols:list[str]):
    ols_formula = f"{y_col} ~ {' * '.join(f'C({item})' for item in x_cols)}"
    #print(ols_formula)
    ols_model = ols(formula=ols_formula, data=df).fit()
    display(sm.stats.anova_lm(ols_model, typ=2))

def manova_test(df: pd.DataFrame, y_cols: list[str], x_col: str):
    manova_formula = f"{' + '.join(y_cols)} ~ {x_col}"
    #print(manova_formula)
    manova_model = MANOVA.from_formula(manova_formula, data=df)
    mv_results = manova_model.mv_test()
    
    dfs = {}
    for factor, result in mv_results.results.items():
        stat_df = pd.DataFrame(result['stat'])
        stat_df.index.name = factor
        dfs[factor] = stat_df.reset_index()
    
    for factor, df_factor in dfs.items():
        print(factor)
        display(df_factor)